In [ ]:
!pip install pm4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pm4py
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/usr/local/lib/python3.12/dist-packages/cudf/pandas/__init__.py:51: UserWarning: Failed to check cudaDevAttrConcurrentManagedAccess with error 35
  warnings.warn(str(e))


In [33]:
# Load dataset
filepath = '/content/UpFLux_Healthcare_Database_labeled.csv'
df = pd.read_csv(filepath, parse_dates=["time:timestamp"])
df['case:concept:name'] = df['case:concept:name'].astype(str)

df

/usr/local/lib/python3.12/dist-packages/cudf/pandas/fast_slow_proxy.py:28: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return fn(*args, **kwargs)


,case:concept:name,Idade,Sexo,Nível de Urgência,CID,Médico Responsável,Doença,Data Inicial,time:timestamp,concept:name,Item,Data Prescrição,Tempo Processo,Setor,Quantidade,Retorno,Multipassante,Convênio,outlier_label
0,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:22,2020-07-21 10:22:00,Atendimento,Atendimento,NaN,NaN,Setor Pronto Atendimento,0.0,Sem retorno,Não,Convênio 9,outlier
1,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:45,2020-07-21 10:49:00,Triagem,Triagem,NaN,NaN,Setor Pronto Atendimento,0.0,Sem retorno,NaN,Convênio 9,outlier
2,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:01,2020-07-21 11:01:00,Exames Laboratoriais,Coronavírus COVID-19 - Diagnóstico Molecular (...,21/07/2020 10:51,9.0,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,outlier
3,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:32,2020-07-21 11:32:00,Consulta,Consulta,NaN,NaN,Setor Pronto Atendimento,0.0,Sem retorno,NaN,Convênio 9,outlier
4,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 19:27,2020-07-21 19:27:00,Alta,Alta para completar tratamento,NaN,NaN,Setor Pronto Atendimento,0.0,Sem retorno,NaN,Convênio 9,outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,Equipo Primeline Air IL (ref.401450p) (cx c/8...,NaN,NaN,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier
3089,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,Cateter Perif. SAFETY 22G (cx c/200) ref. 425...,NaN,NaN,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier
3090,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,Seringa descartavel 10ml Luer Lok (cx c/ 100) ...,NaN,NaN,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier
3091,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,Agulha descartavel 40x12 (cx.c/100) (300017) - BD,NaN,NaN,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier


In [34]:
df.dropna(subset=['time:timestamp'], inplace=True)

start_activities = pm4py.get_start_activities(df, case_id_key="case:concept:name")
end_activities = pm4py.get_end_activities(df, case_id_key="case:concept:name")

print("Start activities: ", start_activities)
print("End activities: ", end_activities)

Start activities:  {'Atendimento': 443}
End activities:  {'Alta': 436, 'Exames Laboratoriais': 3, 'Exames Eletrofisiológicos': 2, 'Exames de Imagem': 2}


In [35]:


# 1. Calcolo il 'start_timestamp' per ogni caso
df['start_timestamp'] = df.groupby('case:concept:name')['time:timestamp'].transform('min')

# 2. Calcolo 'Elapsed Time'
df['Elapsed_Time_Mins'] = (df['time:timestamp'] - df['start_timestamp']).dt.total_seconds() / 60
df['Elapsed_Time_Days'] = df['Elapsed_Time_Mins'] / (60 * 24)

# 3. Calcolo 'Processing Time'
df['Processing_Time'] = df.groupby('case:concept:name')['time:timestamp'].diff().dt.total_seconds() / 60
df['Processing_Time'] = df['Processing_Time'].fillna(0) # Il primo evento ha tempo 0

start_activities_stats = pm4py.get_start_activities(df)
end_activities_stats = pm4py.get_end_activities(df)

df

,case:concept:name,Idade,Sexo,Nível de Urgência,CID,Médico Responsável,Doença,Data Inicial,time:timestamp,concept:name,...,Setor,Quantidade,Retorno,Multipassante,Convênio,outlier_label,start_timestamp,Elapsed_Time_Mins,Elapsed_Time_Days,Processing_Time
0,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:22,2020-07-21 10:22:00,Atendimento,...,Setor Pronto Atendimento,0.0,Sem retorno,Não,Convênio 9,outlier,2020-07-21 10:22:00,0.0,0.000000,0.0
1,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:45,2020-07-21 10:49:00,Triagem,...,Setor Pronto Atendimento,0.0,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,27.0,0.018750,27.0
2,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:01,2020-07-21 11:01:00,Exames Laboratoriais,...,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,39.0,0.027083,12.0
3,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:32,2020-07-21 11:32:00,Consulta,...,Setor Pronto Atendimento,0.0,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,70.0,0.048611,31.0
4,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 19:27,2020-07-21 19:27:00,Alta,...,Setor Pronto Atendimento,0.0,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,545.0,0.378472,475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0
3089,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0
3090,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0
3091,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Setor Pronto Atendimento,1.0,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0


In [36]:
num_events = len(df)
num_cases = len(df['case:concept:name'].unique())
print(f"\nNumber of events: {num_events}\nNumber of cases: {num_cases}")

start_activities = pm4py.get_start_activities(df, case_id_key='case:concept:name', activity_key='concept:name')
end_activities = pm4py.get_end_activities(df, case_id_key='case:concept:name', activity_key='concept:name')
print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))


Number of events: 3075
Number of cases: 443
Start activities: {'Atendimento': 443}
End activities: {'Alta': 436, 'Exames Laboratoriais': 3, 'Exames Eletrofisiológicos': 2, 'Exames de Imagem': 2}


In [37]:
df = df.sort_values(['case:concept:name', 'time:timestamp'])

# 1. Calcolo la differenza di tempo tra una riga e l'altra dello stesso caso
df['processing_time'] = df.groupby('case:concept:name')['time:timestamp'].diff().dt.total_seconds() / 60
df['processing_time'] = df['processing_time'].fillna(0) # Il primo evento è 0

# 2. Calcolo il Lead Time
df['lead_time'] = df.groupby('case:concept:name')['processing_time'].transform('sum')

df.head()

,case:concept:name,Idade,Sexo,Nível de Urgência,CID,Médico Responsável,Doença,Data Inicial,time:timestamp,concept:name,...,Retorno,Multipassante,Convênio,outlier_label,start_timestamp,Elapsed_Time_Mins,Elapsed_Time_Days,Processing_Time,processing_time,lead_time
0,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:22,2020-07-21 10:22:00,Atendimento,...,Sem retorno,Não,Convênio 9,outlier,2020-07-21 10:22:00,0.0,0.000000,0.0,0.0,545.0
1,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:45,2020-07-21 10:49:00,Triagem,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,27.0,0.018750,27.0,27.0,545.0
2,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:01,2020-07-21 11:01:00,Exames Laboratoriais,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,39.0,0.027083,12.0,12.0,545.0
3,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:32,2020-07-21 11:32:00,Consulta,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,70.0,0.048611,31.0,31.0,545.0
4,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 19:27,2020-07-21 19:27:00,Alta,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,545.0,0.378472,475.0,475.0,545.0


In [38]:
# Raggruppo per caso
grouped = df.groupby("case:concept:name")
violations = []

for case_id, group_df in grouped:
    # Ordino per tempo
    group_df = group_df.sort_values("time:timestamp")
    #ultima attività
    last_activity = group_df["concept:name"].iloc[-1]

    # CONTROLLO: Se l'ultima attività NON è 'Alta', è una violazione
    if last_activity != "Alta":
        violations.append({
            "case_id": case_id,
            "last_activity": last_activity
        })

# Risultati
num_violations = len(violations)
total_cases = len(df['case:concept:name'].unique())

print(f"Total cases: {total_cases}")
print(f"Violations (Not ending with Alta): {num_violations}")

if num_violations > 0:
    print("\nExample violations:")
    print(pd.DataFrame(violations).head())

Total cases: 443
Violations (Not ending with Alta): 7

Example violations:
   case_id              last_activity
0  5448329       Exames Laboratoriais
1  5449891  Exames Eletrofisiológicos
2  5451625  Exames Eletrofisiológicos
3  5453991       Exames Laboratoriais
4  5458135           Exames de Imagem


In [39]:
# Funzione per decidere se tenere il caso (DEVE FINIRE CON ALTA)
def keep_trace(group_df):
    # Prende l'ultima attività dopo aver ordinato
    last_act = group_df.sort_values("time:timestamp")["concept:name"].iloc[-1]
    # Ritorna True solo se l'ultima attività è 'Alta'
    return last_act == "Alta"

# Applichiamo il filtro gruppo per gruppo
grouped = df.groupby("case:concept:name", sort=False)
kept_groups = []

for case_id, group_df in grouped:
    if keep_trace(group_df):
        kept_groups.append(group_df)

# Uniamo tutto nel dataframe finale pulito
if kept_groups:
    df = pd.concat(kept_groups, ignore_index=True)

print(f"Final number of cases (conforming to Alta): {len(df['case:concept:name'].unique())}")

df

Final number of cases (conforming to Alta): 436


,case:concept:name,Idade,Sexo,Nível de Urgência,CID,Médico Responsável,Doença,Data Inicial,time:timestamp,concept:name,...,Retorno,Multipassante,Convênio,outlier_label,start_timestamp,Elapsed_Time_Mins,Elapsed_Time_Days,Processing_Time,processing_time,lead_time
0,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:22,2020-07-21 10:22:00,Atendimento,...,Sem retorno,Não,Convênio 9,outlier,2020-07-21 10:22:00,0.0,0.000000,0.0,0.0,545.0
1,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 10:45,2020-07-21 10:49:00,Triagem,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,27.0,0.018750,27.0,27.0,545.0
2,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:01,2020-07-21 11:01:00,Exames Laboratoriais,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,39.0,0.027083,12.0,12.0,545.0
3,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 11:32,2020-07-21 11:32:00,Consulta,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,70.0,0.048611,31.0,31.0,545.0
4,5446538,31,F,Não informado,J111,Médico 72,J11.1 Influenza c/out manif resp dev virus n i...,21/07/2020 19:27,2020-07-21 19:27:00,Alta,...,Sem retorno,NaN,Convênio 9,outlier,2020-07-21 10:22:00,545.0,0.378472,475.0,475.0,545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0,0.0,163.0
2933,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0,0.0,163.0
2934,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0,0.0,163.0
2935,5463012,45,F,Média,N23,Médico 55,N23 Colica nefretica NE,05/08/2020 05:22,2020-08-05 05:22:00,Materiais Hospitalares,...,Sem retorno,NaN,Convênio 9,inlier,2020-08-05 04:59:00,23.0,0.015972,0.0,0.0,163.0


In [51]:
import pm4py
import pandas as pd

df['case:concept:name'] = df['case:concept:name'].astype(str)
df['concept:name'] = df['concept:name'].astype(str)

# NOISE FILTERING
initial_cases = len(df['case:concept:name'].unique())
print(f"Casi prima del filtro rumore: {initial_cases}")

try:
    # Applichiamo il filtro direttamente. Manteniamo le varianti che coprono il 98% dei casi.
    filtered_df = pm4py.filter_variants_by_coverage_percentage(
        df,
        0.98,
        case_id_key='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # check
    if len(filtered_df) > 0 and len(filtered_df) < len(df):
        df = filtered_df
        print("Filtro rumore applicato con successo.")
    elif len(filtered_df) == len(df):
        print("Il filtro non ha trovato varianti abbastanza rare da rimuovere (soglia 98%). Dati mantenuti.")
    else:
        print("ATTENZIONE: Il filtro è stato troppo aggressivo. Mantengo i dati originali.")

except Exception as e:
    print(f"Impossibile applicare il filtro varianti ({e}). Procedo con il dataset attuale (che è comunque valido).")

final_cases = len(df['case:concept:name'].unique())
print(f"\n--- RISULTATI FINALI ---")
print(f"Casi Totali: {final_cases}")

# Salvataggio
output_filename = '27_11_Filtered_Log.csv'
df.to_csv(output_filename, index=False)
print(f"Dataset salvato come: '{output_filename}'")


Casi prima del filtro rumore: 436
ATTENZIONE: Il filtro è stato troppo aggressivo. Mantengo i dati originali.

--- RISULTATI FINALI ---
Casi Totali: 436
Dataset salvato come: '27_11_Filtered_Log.csv'
